In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
data = pd.read_csv('/content/IMDB.csv', sep = ',', engine='python',encoding='utf-8', error_bad_lines=True)

<ipython-input-2-7025710ea99b>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/content/IMDB.csv', sep = ',', engine='python',encoding='utf-8', error_bad_lines=True)


In [3]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
from platform import python_version
import numpy, matplotlib, pandas, torch, torchtext

print("python==" + python_version())
print("numpy==" + numpy.__version__)
print("torch==" + torch.__version__)
print("torchtext==" + torchtext.__version__)
print("matplotlib==" + matplotlib.__version__)

python==3.10.12
numpy==1.23.5
torch==2.1.0+cu118
torchtext==0.16.0+cpu
matplotlib==3.7.1


In [5]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and temporary (validation + test) sets
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# Split the temporary data into validation and test sets
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [6]:
train_data.columns

Index(['review', 'sentiment'], dtype='object')

In [7]:
train_data.iloc[0]

review       That's what I kept asking myself during the ma...
sentiment                                             negative
Name: 39087, dtype: object

In [8]:
train_data.shape, validation_data.shape, test_data.shape

((40000, 2), (5000, 2), (5000, 2))

In [9]:
!pip install portalocker>=2.0.0

In [11]:
type(train_data.review[0])

str

In [12]:
example_text=train_data.review[0]
example_text

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [13]:
text = re.sub("<[^>]*>", "", example_text)
text

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [14]:
text = text.lower()
text

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [15]:
emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
emoticons

[]

In [16]:
text = re.sub("[\W]+", " ", text)
text

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dare forget pretty

In [17]:
def tokenizer(text):
    # step 1. remove HTML tags. they are not helpful in understanding the sentiments of a review
    # step 2: use lowercase for all text to keep symmetry
    # step 3: extract emoticons. keep them as they are important sentiment signals
    # step 4: remove punctuation marks
    # step 5: put back emoticons
    # step 6: generate word tokens
    text = re.sub("<[^>]*>", "", text)
    text = text.lower()
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = re.sub("[\W]+", " ", text)
    text = text + " ".join(emoticons).replace("-", "")
    tokenized = text.split()
    return tokenized

In [18]:
example_tokens = tokenizer(example_text)
example_tokens

['one',
 'of',
 'the',
 'other',
 'reviewers',
 'has',
 'mentioned',
 'that',
 'after',
 'watching',
 'just',
 '1',
 'oz',
 'episode',
 'you',
 'll',
 'be',
 'hooked',
 'they',
 'are',
 'right',
 'as',
 'this',
 'is',
 'exactly',
 'what',
 'happened',
 'with',
 'me',
 'the',
 'first',
 'thing',
 'that',
 'struck',
 'me',
 'about',
 'oz',
 'was',
 'its',
 'brutality',
 'and',
 'unflinching',
 'scenes',
 'of',
 'violence',
 'which',
 'set',
 'in',
 'right',
 'from',
 'the',
 'word',
 'go',
 'trust',
 'me',
 'this',
 'is',
 'not',
 'a',
 'show',
 'for',
 'the',
 'faint',
 'hearted',
 'or',
 'timid',
 'this',
 'show',
 'pulls',
 'no',
 'punches',
 'with',
 'regards',
 'to',
 'drugs',
 'sex',
 'or',
 'violence',
 'its',
 'is',
 'hardcore',
 'in',
 'the',
 'classic',
 'use',
 'of',
 'the',
 'word',
 'it',
 'is',
 'called',
 'oz',
 'as',
 'that',
 'is',
 'the',
 'nickname',
 'given',
 'to',
 'the',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'it',
 'focuses',
 'mainly',
 'o

In [19]:
from collections import Counter

token_counts = Counter()
token_counts.update(example_tokens)
token_counts

Counter({'one': 1,
         'of': 7,
         'the': 16,
         'other': 2,
         'reviewers': 1,
         'has': 1,
         'mentioned': 1,
         'that': 4,
         'after': 1,
         'watching': 2,
         'just': 2,
         '1': 1,
         'oz': 6,
         'episode': 2,
         'you': 3,
         'll': 3,
         'be': 2,
         'hooked': 1,
         'they': 1,
         'are': 2,
         'right': 2,
         'as': 4,
         'this': 3,
         'is': 9,
         'exactly': 1,
         'what': 2,
         'happened': 1,
         'with': 5,
         'me': 4,
         'first': 2,
         'thing': 1,
         'struck': 2,
         'about': 1,
         'was': 3,
         'its': 2,
         'brutality': 1,
         'and': 6,
         'unflinching': 1,
         'scenes': 1,
         'violence': 4,
         'which': 1,
         'set': 1,
         'in': 3,
         'from': 1,
         'word': 2,
         'go': 1,
         'trust': 1,
         'not': 3,
         'a': 3,

In [20]:
sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
sorted_by_freq_tuples

[('the', 16),
 ('is', 9),
 ('of', 7),
 ('oz', 6),
 ('and', 6),
 ('to', 6),
 ('it', 6),
 ('i', 6),
 ('with', 5),
 ('for', 5),
 ('that', 4),
 ('as', 4),
 ('me', 4),
 ('violence', 4),
 ('you', 3),
 ('ll', 3),
 ('this', 3),
 ('was', 3),
 ('in', 3),
 ('not', 3),
 ('a', 3),
 ('show', 3),
 ('or', 3),
 ('on', 3),
 ('prison', 3),
 ('so', 3),
 ('t', 3),
 ('forget', 3),
 ('other', 2),
 ('watching', 2),
 ('just', 2),
 ('episode', 2),
 ('be', 2),
 ('are', 2),
 ('right', 2),
 ('what', 2),
 ('first', 2),
 ('struck', 2),
 ('its', 2),
 ('word', 2),
 ('city', 2),
 ('where', 2),
 ('high', 2),
 ('more', 2),
 ('away', 2),
 ('say', 2),
 ('due', 2),
 ('but', 2),
 ('who', 2),
 ('inmates', 2),
 ('get', 2),
 ('one', 1),
 ('reviewers', 1),
 ('has', 1),
 ('mentioned', 1),
 ('after', 1),
 ('1', 1),
 ('hooked', 1),
 ('they', 1),
 ('exactly', 1),
 ('happened', 1),
 ('thing', 1),
 ('about', 1),
 ('brutality', 1),
 ('unflinching', 1),
 ('scenes', 1),
 ('which', 1),
 ('set', 1),
 ('from', 1),
 ('go', 1),
 ('trust', 1),

In [21]:
##
# step 1: convert our sorted list of tokens to OrderedDict
from collections import OrderedDict

ordered_dict = OrderedDict(sorted_by_freq_tuples)
ordered_dict

OrderedDict([('the', 16),
             ('is', 9),
             ('of', 7),
             ('oz', 6),
             ('and', 6),
             ('to', 6),
             ('it', 6),
             ('i', 6),
             ('with', 5),
             ('for', 5),
             ('that', 4),
             ('as', 4),
             ('me', 4),
             ('violence', 4),
             ('you', 3),
             ('ll', 3),
             ('this', 3),
             ('was', 3),
             ('in', 3),
             ('not', 3),
             ('a', 3),
             ('show', 3),
             ('or', 3),
             ('on', 3),
             ('prison', 3),
             ('so', 3),
             ('t', 3),
             ('forget', 3),
             ('other', 2),
             ('watching', 2),
             ('just', 2),
             ('episode', 2),
             ('be', 2),
             ('are', 2),
             ('right', 2),
             ('what', 2),
             ('first', 2),
             ('struck', 2),
             ('its', 2),
        

In [22]:
##
# Check the length of our dictionary
len(ordered_dict)

189

In [23]:
##
# step 2: convert the ordered dict to torchtext.vocab
from torchtext.vocab import vocab

vb = vocab(ordered_dict)
vb.get_stoi()

{'darker': 187,
 'your': 186,
 'touch': 185,
 'thats': 182,
 'uncomfortable': 180,
 'experience': 176,
 'skills': 175,
 'street': 174,
 'lack': 173,
 'their': 172,
 'bitches': 171,
 'being': 168,
 'may': 177,
 'class': 167,
 'mannered': 165,
 'well': 164,
 'order': 163,
 'out': 160,
 'guards': 158,
 'levels': 154,
 'got': 152,
 'developed': 150,
 'ready': 148,
 'couldn': 147,
 'surreal': 146,
 'become': 178,
 'saw': 144,
 'ever': 143,
 'comfortable': 179,
 'around': 142,
 'mess': 141,
 'doesn': 140,
 'romance': 139,
 'charm': 138,
 'audiences': 137,
 'nickel': 161,
 'pictures': 134,
 'pretty': 133,
 'dare': 132,
 'kill': 162,
 'wouldn': 131,
 'goes': 129,
 'appeal': 127,
 'after': 55,
 'more': 43,
 'has': 53,
 'inwards': 103,
 'where': 41,
 'say': 45,
 'oz': 3,
 'turned': 169,
 'sold': 159,
 'that': 10,
 'what': 35,
 'right': 34,
 'one': 51,
 'irish': 115,
 'i': 7,
 'pulls': 74,
 'was': 17,
 'scenes': 65,
 'reviewers': 52,
 'and': 4,
 't': 26,
 'maximum': 87,
 'just': 30,
 'many': 108,

In [24]:
train_data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [25]:
##
# step 1: convert reviews into tokens
# step 2: find frequency of tokens

token_counts = Counter()

for i in range(0,len(train_data)):
    tokens = tokenizer(train_data['review'].iloc[i])
    token_counts.update(tokens)

print('message vocab size:', len(token_counts))

message vocab size: 94975


In [26]:
token_counts

Counter({'that': 115208,
         's': 99834,
         'what': 25950,
         'i': 140753,
         'kept': 1204,
         'asking': 375,
         'myself': 1848,
         'during': 3442,
         'the': 534382,
         'many': 10733,
         'fights': 435,
         'screaming': 402,
         'matches': 155,
         'swearing': 89,
         'and': 259123,
         'general': 1211,
         'mayhem': 134,
         'permeate': 12,
         '84': 33,
         'minutes': 4692,
         'comparisons': 95,
         'also': 14435,
         'stand': 1240,
         'up': 20988,
         'when': 22267,
         'you': 55158,
         'think': 11500,
         'of': 231147,
         'one': 42939,
         'dimensional': 408,
         'characters': 11485,
         'who': 33815,
         'have': 44185,
         'so': 32758,
         'little': 9924,
         'depth': 808,
         'it': 152778,
         'is': 168846,
         'virtually': 333,
         'impossible': 763,
         'to': 214267,
  

In [27]:
##
# step 3: sort the token based on their frequency
# step 4: put the sorted tokens in OrderedDict
# step 5: convert token to integers using vocab object

sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)

vb = vocab(ordered_dict)

vb.insert_token("<pad>", 0)  # special token for padding
vb.insert_token("<unk>", 1)  # special token for unknown words
vb.set_default_index(1)

# print some token indexes from vocab
for token in ["this", "is", "an", "example"]:
    print(token, " --> ", vb[token])

this  -->  11
is  -->  7
an  -->  35
example  -->  472


In [28]:
len(vb.get_itos())

94977

In [29]:
vb.get_itos()[:10]

['<pad>', '<unk>', 'the', 'and', 'a', 'of', 'to', 'is', 'it', 'in']

In [30]:
vb.get_itos()[54]

'when'

In [31]:
vb.get_itos()[-10:]

['unarticulated',
 'catfighting',
 'smethurst',
 'villein',
 'tine',
 'trustment',
 'coooool',
 'tunnelful',
 'hieronymus',
 'wilshire']

In [32]:
##
# inline lambda functions for text and label precessing
text_pipeline = lambda x: [vb[token] for token in tokenizer(x)]
label_pipeline = lambda x: 1.0 if x == "positive" else 0.0

In [33]:
example_text

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [34]:
##
# apply text_pipeline to example_text
text_pipeline(example_text)

[29,
 5,
 2,
 81,
 2074,
 47,
 1060,
 12,
 101,
 149,
 42,
 303,
 3058,
 391,
 21,
 233,
 30,
 3311,
 33,
 25,
 204,
 15,
 11,
 7,
 632,
 48,
 595,
 18,
 69,
 2,
 88,
 150,
 12,
 3149,
 69,
 45,
 3058,
 14,
 94,
 5742,
 3,
 14741,
 135,
 5,
 571,
 62,
 268,
 9,
 204,
 38,
 2,
 651,
 142,
 1734,
 69,
 11,
 7,
 24,
 4,
 120,
 17,
 2,
 7797,
 2238,
 41,
 11502,
 11,
 120,
 2621,
 57,
 5921,
 18,
 5654,
 6,
 1444,
 378,
 41,
 571,
 94,
 7,
 3784,
 9,
 2,
 360,
 364,
 5,
 2,
 651,
 8,
 7,
 441,
 3058,
 15,
 12,
 7,
 2,
 10750,
 359,
 6,
 2,
 15586,
 6833,
 2486,
 1041,
 48744,
 8,
 2759,
 1428,
 22,
 21823,
 523,
 35,
 4614,
 2487,
 5,
 2,
 1243,
 116,
 31,
 2,
 7275,
 28,
 2963,
 11483,
 3,
 397,
 48745,
 37,
 17479,
 7,
 24,
 300,
 22,
 2,
 4720,
 2995,
 523,
 7,
 344,
 6,
 109,
 22748,
 7694,
 37553,
 13598,
 5118,
 8081,
 2557,
 3,
 52,
 37,
 41976,
 328,
 9164,
 7059,
 12513,
 3,
 8405,
 29282,
 25,
 112,
 224,
 242,
 10,
 61,
 133,
 2,
 280,
 1321,
 5,
 2,
 120,
 7,
 688,
 6,
 2,
 195

In [35]:
train_data_vect = []

for i in range(0,len(train_data['review'])):
  out = text_pipeline(train_data.review.iloc[i])
  train_data_vect.append(out)

In [36]:
train_data_vect[0]

[12,
 13,
 48,
 10,
 797,
 2163,
 534,
 306,
 2,
 109,
 1900,
 2037,
 4353,
 6530,
 3,
 792,
 4862,
 12,
 22053,
 2,
 12616,
 229,
 2,
 6202,
 82,
 775,
 56,
 54,
 21,
 102,
 5,
 2,
 29,
 2013,
 103,
 36,
 28,
 37,
 121,
 1121,
 12,
 8,
 7,
 2367,
 1185,
 6,
 458,
 48,
 550,
 6,
 93,
 33,
 25,
 42,
 905,
 405,
 28486,
 17,
 2,
 157,
 6,
 2981,
 27,
 36139,
 4390,
 22,
 4,
 2943,
 12,
 47,
 76,
 222,
 74,
 128,
 9,
 81,
 3152,
 197,
 22,
 238,
 3,
 2,
 435,
 10,
 202,
 4963,
 10,
 141,
 24,
 65,
 29,
 17,
 14598,
 75,
 366,
 306,
 4,
 20,
 19,
 8,
 202,
 30,
 309,
 12,
 57852,
 40320,
 15,
 2,
 1858,
 13,
 10158,
 117,
 421,
 3,
 46536,
 46537,
 15,
 2,
 1685,
 11318,
 572,
 72,
 427,
 381,
 10,
 87,
 23,
 119,
 48,
 115,
 369,
 33,
 11930,
 38,
 19,
 46,
 10,
 14,
 93,
 10,
 227,
 6868,
 17,
 4,
 365,
 12367,
 1165,
 24136,
 64,
 57853,
 57854,
 9,
 2,
 469,
 217,
 1007,
 6,
 4194,
 9,
 4,
 178,
 5,
 37,
 441,
 689,
 617,
 12,
 71,
 233,
 239,
 112,
 842,
 38,
 173,
 32,
 221,
 12,
 13

In [38]:
train_data.shape, validation_data.shape, test_data.shape

((40000, 2), (5000, 2), (5000, 2))

In [39]:
train_data.columns

Index(['review', 'sentiment'], dtype='object')

In [37]:
 validation_data.shape, test_data.shape

((5000, 2), (5000, 2))

In [40]:
train_data = [(label, data) for label, data in zip(train_data['sentiment'], train_data['review'])]

In [41]:
validation_data = [(label, data) for label, data in zip(validation_data['sentiment'], validation_data['review'])]

In [42]:
test_data = [(label, data) for label, data in zip(test_data['sentiment'], test_data['review'])]

In [43]:
train_data[0]

('negative',
 'That\'s what I kept asking myself during the many fights, screaming matches, swearing and general mayhem that permeate the 84 minutes. The comparisons also stand up when you think of the one-dimensional characters, who have so little depth that it is virtually impossible to care what happens to them. They are just badly written cyphers for the director to hang his multicultural beliefs on, a topic that has been done much better in other dramas both on TV and the cinema.<br /><br />I must confess, I\'m not really one for spotting bad performances during a film, but it must be said that Nichola Burley (as the heroine\'s slutty best friend) and Wasim Zakir (as the nasty, bullying brother) were absolutely terrible. I don\'t know what acting school they graduated from, but if I was them I\'d apply for a full refund post haste. Only Samina Awan in the lead role manages to impress in a cast of so-called British talent that we\'ll probably never hear from again. At least, that\'

In [44]:
validation_data[0]

('negative',
 'Hollywood has churned out yet another garbage that\'s wildly overhyped and underwhelming on a first-time viewing basis. Hannibal is bad, terrible, inept, lame, droll, idiotic, contrived, laughable and utterly atrocious (no pun intended). Minor spoilers follow...<br /><br />This movie has huge logic holes - more than any Bruckheimer/Bay movie - or for that matter - any movie that exemplify the indulgence of Hollywood exaggeration. It\'s a slick Hollywood production designed to cash in on Hannibal Lector mania, directed by "so-somber-he-takes-this-way-too-seriously" hack director Ridley Scott and produced by a hack Italian producer with an inflated ego whose credo is "doesn\'t matter whether film is s**t, money is good".<br /><br />I can\'t get over the fact that acclaimed screenwriters David Mamet and Steven Zaillian wrote this tripe adapted from a lame and pretentious book by a good-novelist-turned-hack-author Thomas Harris. David and Steven - well-known and immensely ta

In [45]:
test_data[0]

('negative',
 'the tortuous emotional impact is degrading, whether adult or adolescent the personal values shown in this movie belong in a bad psychodrama if anywhere at all. This movie has a plot, but it is all evil from start to end. This is no way for people to act and degrades both sexes all the way through the movie. teen killing - bad preteen sex - bad emotional battering - bad animal cruelty - bad psychological torture - bad parental neglect - bad the only merit if any is the excellent color shots of contrasting red, blond and green leaves a bad feeling for anyone that respects life and peace, what a bad mistake to make, or to watch... it is UGLY')

In [46]:
##
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [47]:
##
# a function to apply pre-processing steps at a batch level
import torch.nn as nn

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []

    # iterate over all reviews in a batch
    for _label, _text in batch:
        # label preprocessing
        label_list.append(label_pipeline(_label))
        # text preprocessing
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)

        # store the processed text in a list
        text_list.append(processed_text)

        # store the length of processed text
        # this will come handy in future when we want to know the original size of a text (without padding)
        lengths.append(processed_text.size(0))

    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)

    # pad the processed reviews to make their lengths consistant
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list, batch_first=True)

    # return
    # 1. a list of processed and padded review texts
    # 2. a list of processed labels
    # 3. a list of review text original lengths (before padding)
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

In [48]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    train_data, batch_size=4, shuffle=False, collate_fn=collate_batch
)
text_batch, label_batch, length_batch = next(iter(dataloader))

print("text_batch.shape: ", text_batch.shape)
print("label_batch: ", label_batch)
print("length_batch: ", length_batch)

text_batch.shape:  torch.Size([4, 417])
label_batch:  tensor([0., 0., 1., 0.], device='cuda:0')
length_batch:  tensor([417, 279,  93, 183], device='cuda:0')


In [49]:
print(text_batch[2])

tensor([    4,  1332,   113,    63,  2839,     5,     9,     2,  1271,    17,
          113,  4588,  2596,    22,  1702,  3956,     3,    85,    11,     7,
          334,    34,  4236,    79,     6,  5747,  1344,     6,   255,    81,
            2,    63,  2759,    22,     4, 17354,    36,   491,    37,    74,
            6,    30,     4,  2219,  1621,    15,    70,    15,     4,    50,
          603,     3,   330,     4,  6820,  1345,     7,    53,  3544,     6,
           89,    15,     2,    63,  4237,    71,    66,     2,  1333,  1777,
            2,  2303,     9,    27,   819,   293,  1511,     3,    85,    26,
        33012,    18,     2,  1515, 12368,    12,   397,    89,     4,   317,
            3,   704,    63,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [50]:
batch_size = 32

train_dl = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_batch
)
valid_dl = DataLoader(
    validation_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch
)
test_dl = DataLoader(
    test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch
)

In [51]:
##
# model training pipeline
# https://github.com/rasbt/machine-learning-book/blob/main/ch15/ch15_part2.ipynb
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred >= 0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item() * label_batch.size(0)
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)


# model evaluation pipeline
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred >= 0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item() * label_batch.size(0)
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

In [52]:
##
# https://github.com/rasbt/machine-learning-book/blob/main/ch15/ch15_part2.ipynb
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(
            out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True
        )
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [53]:
vocab_size = len(vb)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size)
model = model.to(device)

In [54]:
torch.manual_seed(1)

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [56]:
from tqdm import tqdm

In [57]:
num_epochs = 10
for epoch in tqdm(range(num_epochs),desc='processing'):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(
        f"Epoch {epoch} train accuracy: {acc_train:.4f}; val accuracy: {acc_valid:.4f}"
    )

processing:  10%|█         | 1/10 [00:56<08:26, 56.29s/it]

Epoch 0 train accuracy: 0.8163; val accuracy: 0.7080


processing:  20%|██        | 2/10 [01:53<07:33, 56.70s/it]

Epoch 1 train accuracy: 0.8496; val accuracy: 0.8672


processing:  30%|███       | 3/10 [02:49<06:36, 56.58s/it]

...

Epoch 7 train accuracy: 0.9778; val accuracy: 0.8942


processing:  90%|█████████ | 9/10 [08:22<00:55, 55.46s/it]

Epoch 8 train accuracy: 0.9845; val accuracy: 0.8948


processing: 100%|██████████| 10/10 [09:17<00:00, 55.72s/it]

Epoch 9 train accuracy: 0.9889; val accuracy: 0.8906


In [62]:
def classify_review(text):
    text_list, lengths = [], []

    # process review text with text_pipeline
    # note: "text_pipeline" has dependency on data vocabulary
    processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
    text_list.append(processed_text)

    # get processed review tokens length
    lengths.append(processed_text.size(0))
    lengths = torch.tensor(lengths)

    # change the dimensions from (torch.Size([8]), torch.Size([1, 8]))
    # nn.utils.rnn.pad_sequence(text_list, batch_first=True) does this too
    padded_text_list = torch.unsqueeze(processed_text, 0)

    # move tensors to correct device
    padded_text_list = padded_text_list.to(device)
    lengths = lengths.to(device)

    # get prediction
    model.eval()
    pred = model(padded_text_list, lengths)
    print("model pred: ", pred)

    # positive or negative review
    review_class = 'negative' # else case
    if (pred>=0.5) == 1:
        review_class = "positive"

    print("review type: ", review_class)

In [63]:
##
# create two random texts with strong positive and negative sentiments
pos_review = 'i love this movie. it was so good.'
neg_review = 'slow and boring. waste of time.'

In [60]:
classify_review(pos_review)

model pred:  tensor([[0.9024]], device='cuda:0', grad_fn=<SigmoidBackward0>)
review type:  positive


In [61]:
classify_review(neg_review)

model pred:  tensor([[0.0025]], device='cuda:0', grad_fn=<SigmoidBackward0>)
review type:  negative
